<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Rag_Fusion_Pipeline_PostgreSQL_Embedding_CLAUDE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Fusion Query Pipeline

This notebook shows how to implement RAG Fusion using the LlamaIndex Query Pipeline syntax.

In [1]:
!nvidia-smi

Fri May 31 21:09:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Required Dependencies

In [2]:
#added by Frank Morales(FM) 11/01/2024
%pip install openai  --root-user-action=ignore -q
!pip install llama_index phoenix pyvis network -q
!pip install llama_hub -q
%pip install colab-env --upgrade --quiet --root-user-action=ignore
!pip install accelerate -q
#!pip install typing_extensions

!pip install langchain --quiet
!pip install accelerate --quiet
!pip install transformers --quiet
!pip install bitsandbytes --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

## Setup / Load Data

We load in the pg_essay.txt data.

In [3]:
import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
%cd /content/
!wget "https://www.dropbox.com/s/f6bmb19xdg0xedm/paul_graham_essay.txt?dl=1" -O pg_essay.txt
#!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt' -O pg_essay.txt

Mounted at /content/gdrive
/content
--2024-05-31 21:13:05--  https://www.dropbox.com/s/f6bmb19xdg0xedm/paul_graham_essay.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /scl/fi/f2hfma1kcravwajwskek7/paul_graham_essay.txt?rlkey=kbj5qtm5c8bf1sndadp2rzrse&dl=1 [following]
--2024-05-31 21:13:05--  https://www.dropbox.com/scl/fi/f2hfma1kcravwajwskek7/paul_graham_essay.txt?rlkey=kbj5qtm5c8bf1sndadp2rzrse&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5c91ad66f27f56fa9693c88055.dl.dropboxusercontent.com/cd/0/inline/CT_6tInyOmrtFyxBxDe3wkMEaPTZ4S6hQ5Anb7X7xazrbEQpvELDgWASzQt2UwH0_HfqfLRlxjdekzooBsNC-HTFbcrqHGW5ysMJKgIUzWB-qH23c7ZZ157IB00-_XFwWzs/file?dl=1# [following]
--2024-05-31 21:13:06--  https://uc5c91ad66f27f56fa9693c880

POSTGRESQL
https://www.atlantic.net/dedicated-server-hosting/how-to-install-and-configure-postgres-14-on-ubuntu/

In [4]:
#ADDED By FM 31/05/2024
!apt-get update -y
!apt-get install postgresql-14 -y
!service postgresql restart
!sudo apt install postgresql-server-dev-all

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,084 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,377 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchpadcontent.ne

In [5]:
#ADDED By FM 11/01/2024

# install PSQL WITH DEV Libraries AND PGVECTOR
#!apt install postgresql postgresql-contrib &>log
#!service postgresql restart
#!sudo apt install postgresql-server-dev-all

In [6]:
#!sudo apt install postgresql-server-dev-14

In [7]:
print()
# PostGRES SQL Settings
%cd /content/
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

print('START: PG embedding COMPILATION')
%cd /content/
!git clone https://github.com/neondatabase/pg_embedding.git
%cd /content/pg_embedding
!make
!sudo make install # may need sudo
print('END: PG embedding COMPILATION')
print()

#!sudo -u postgres psql -c "DROP EXTENSION embedding"
!sudo -u postgres psql -c "CREATE EXTENSION embedding"
#!sudo -u postgres psql -c "DROP TABLE documents"
!sudo -u postgres psql -c "CREATE TABLE documents(id integer PRIMARY KEY, embedding real[])"


/content
ALTER ROLE
START: PG embedding COMPILATION
/content
Cloning into 'pg_embedding'...
remote: Enumerating objects: 553, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 553 (delta 140), reused 135 (delta 106), pack-reused 370
Receiving objects: 100% (553/553), 270.29 KiB | 22.52 MiB/s, done.
Resolving deltas: 100% (317/317), done.
/content/pg_embedding
gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -g -g -O2 -flto=auto -ffat-lto-objects -flto=auto -ffat-lto-objects -fstack-protector-strong -Wformat -Werror=format-security -fno-omit-frame-pointer -Ofast -fPIC -I. -I./ -I/usr/include/postgresql/14/server -I/usr/include/postgresql/internal  -Wdate-time -D_FORTIFY_SOU

In [8]:
#%pip install openai --root-user-action=ignore
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
#print(os.getenv("OPENAI_API_KEY"))

from llama_index.core import SimpleDirectoryReader
reader = SimpleDirectoryReader(input_files=["/content/pg_essay.txt"])
docs = reader.load_data()

In [9]:
%pip install -U langchain-community -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.4 MB/s eta 0:00:00


In [10]:
#ADDED By FM 11/01/2024

from typing import List, Tuple
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import PGEmbedding

loader = TextLoader("/content/pg_essay.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs0 = text_splitter.split_documents(documents)

collection_name0 = "pg_essay"
print(f'# of Document Pages {len(documents)}')
print(f'# of Document Chunks: {len(docs0)}')

# of Document Pages 1
# of Document Chunks: 100


## Setup Llama Pack



In [11]:
#!pip install llama_index
import llama_index
print('LLAMA INDEX VERSION: %s'%llama_index.core.__version__)
#llama_index.core.
from llama_index.core.query_pipeline import QueryPipeline
import llama_index.core.query_pipeline as query_pipeline
#llama_index.core

LLAMA INDEX VERSION: 0.10.42


In [12]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

In [13]:
from llama_index.core.query_pipeline import QueryPipeline
from llama_index.core import PromptTemplate

# try chaining basic prompts
prompt_str = "Please generate related movies to {movie_name}"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model="gpt-3.5-turbo")

p = QueryPipeline(chain=[prompt_tmpl, llm], verbose=True)

In [14]:
output = p.run(movie_name="The Departed")

> Running module 8982f4bd-afd3-4055-b5b0-aae630217728 with input: 
movie_name: The Departed

> Running module a98a9523-eca5-48ac-84b5-6466fa63c2f2 with input: 
messages: Please generate related movies to The Departed



In [15]:
print(str(output))

assistant: 1. Infernal Affairs (2002) - The original Hong Kong film that inspired The Departed
2. The Town (2010) - A crime thriller directed by Ben Affleck
3. Goodfellas (1990) - Martin Scorsese's classic mobster film
4. The Godfather (1972) - Francis Ford Coppola's iconic mafia saga
5. Mystic River (2003) - Another crime drama directed by Clint Eastwood
6. Casino (1995) - Martin Scorsese's film about the mob in Las Vegas
7. American Gangster (2007) - Ridley Scott's crime drama based on a true story
8. The Departed (2006) - The Departed is a 2006 American crime film directed by Martin Scorsese and written by William Monahan. It is a remake of the 2002 Hong Kong film Infernal Affairs. The film stars Leonardo DiCaprio, Matt Damon, Jack Nicholson, and Mark Wahlberg.
9. The Irishman (2019) - Martin Scorsese's epic crime drama starring Robert De Niro, Al Pacino, and Joe Pesci
10. Donnie Brasco (1997) - A true story about an undercover FBI agent infiltrating the mob


In [16]:
# Option 1: Use `download_llama_pack`
# from llama_index.llama_pack import download_llama_pack

# RAGFusionPipelinePack = download_llama_pack(
#     "RAGFusionPipelinePack",
#     "./rag_fusion_pipeline_pack",
#     # leave the below line commented out if using the notebook on main
#     # llama_hub_url="https://raw.githubusercontent.com/run-llama/llama-hub/jerry/add_query_pipeline_pack/llama_hub"
# )

# Option 2: Import from llama_hub package
#RAGFusionPipelinePack                                           RAGFusionPipelinePack
#from llama_hub.llama_packs.query.rag_fusion_pipeline.base import RAGFusionPipelinePack
#from llama_index.llms import OpenAI

# CLAUDE3 - MODEL


In [17]:
%pip install llama-index-llms-anthropic -q
!pip install datasets pandas anthropic openai colab-env -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.1/869.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00


In [18]:
import anthropic
import os
import colab_env
import json

api_key = os.environ["CLAUDE3_API_KEY"]
model="claude-3-opus-20240229"

client = anthropic.Anthropic(
    api_key=api_key,
)

In [19]:
from llama_index.llms.anthropic import Anthropic

# To customize your API key, do this
# otherwise it will lookup ANTHROPIC_API_KEY from your env variable
# llm = Anthropic(api_key="")
llm = Anthropic(api_key=api_key,model="claude-3-opus-20240229")

In [20]:
resp_claude3 = llm.complete("Paul Graham is ")

In [21]:
print(resp_claude3)

Paul Graham is an English-born American computer scientist, entrepreneur, venture capitalist, and essayist. He is best known for his work on the programming language Lisp, his former startup Viaweb (which was later acquired by Yahoo! and became Yahoo! Store), and for co-founding the influential startup accelerator Y Combinator. Here are some key points about Paul Graham:

1. Programming: Graham is a proponent of the Lisp programming language and has written several books on the topic, including "On Lisp" (1993) and "ANSI Common Lisp" (1995).

2. Viaweb: In 1995, Graham and Robert Morris founded Viaweb, the first software as a service (SaaS) company. Viaweb's software allowed users to create their own online stores. In 1998, Viaweb was acquired by Yahoo! for $49.6 million.

3. Y Combinator: In 2005, Graham co-founded Y Combinator, a startup accelerator that provides seed funding, advice, and connections to early-stage startups. Y Combinator has been influential in the startup world and 

CLAUDE3 for FILMS

In [22]:
p = QueryPipeline(chain=[prompt_tmpl, llm], verbose=True)
output = p.run(movie_name="The Departed")

> Running module c8a4428e-e8a4-43fd-a798-6713c66250ed with input: 
movie_name: The Departed

> Running module 4ffc1c95-0103-438b-a743-66e283d66b56 with input: 
messages: Please generate related movies to The Departed



In [23]:
print(str(output))

assistant: Here are some movies related to The Departed:

1. Goodfellas (1990) - Another Martin Scorsese crime drama featuring similar themes and style.

2. Infernal Affairs (2002) - The Hong Kong film that inspired The Departed.

3. Heat (1995) - A crime thriller about a detective chasing a professional thief, featuring themes of loyalty and betrayal.

4. Donnie Brasco (1997) - Based on a true story, an undercover FBI agent infiltrates the mob.

5. The Town (2010) - Directed by and starring Ben Affleck, a crime drama set in Boston involving robberies and an undercover FBI agent.

6. Black Mass (2015) - Another Boston-based crime drama, focusing on the infamous Irish mob boss Whitey Bulger, played by Johnny Depp.

7. Reservoir Dogs (1992) - Quentin Tarantino's debut film about a group of thieves and their loyalty to each other.

8. Carlito's Way (1993) - A crime drama directed by Brian De Palma, featuring Al Pacino as an ex-convict trying to go straight.

9. American Gangster (2007) - 

# EMBEDDING

In [24]:
#!pip install -U langchain-community -q

In [25]:
# 20x faster than pgvector: introducing pg_embedding extension for vector search in Postgres and LangChain
# https://neon.tech/blog/pg-embedding-extension-for-vector-search

#ADDED By FM 11/01/2024

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import PGEmbedding

# https://supabase.com/blog/fewer-dimensions-are-better-pgvector
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

collection_name='Paul Graham Essay'
connection_string = os.getenv("DATABASE_URL")

db = PGEmbedding.from_documents(
    embedding=embeddings,
    documents=docs0,
    collection_name=collection_name,
    connection_string=connection_string,
)

#db.create_hnsw_index(dims = 1536, m = 8, ef_construction = 16, ef_search = 16)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [26]:
#ADDED By FM 11/01/2024
query='What did the author do growing up?'
docs_with_score: List[Tuple[Document, float]] = db.similarity_search_with_score(query)

print()
print(query)
print()

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)


What did the author do growing up?

--------------------------------------------------------------------------------
Score:  0.5991553
What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fl

# RAG FUSION PIPELINE

In [27]:
"""RAG Fusion Pipeline."""

from typing import Any, Dict, List, Optional

from llama_index.core import Document, ServiceContext, VectorStoreIndex
from llama_index.core.llama_pack.base import BaseLlamaPack
from llama_index.core.llms.llm import LLM
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.query_pipeline.components.argpacks import ArgPackComponent
from llama_index.core.query_pipeline.components.function import FnComponent
from llama_index.core.query_pipeline.components.input import InputComponent
from llama_index.core.query_pipeline.query import QueryPipeline
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.schema import NodeWithScore
from llama_index.llms.openai import OpenAI

DEFAULT_CHUNK_SIZES = [128, 256, 512, 1024]


def reciprocal_rank_fusion(
    results: List[List[NodeWithScore]],
) -> List[NodeWithScore]:
    """Apply reciprocal rank fusion.

    The original paper uses k=60 for best results:
    https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    """
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}
    rank=0

#for rank, node_with_score in enumerate(
#            sorted(nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True)
#        ):

# The above lines commented generated this error AttributeError: 'tuple' object has no attribute 'score'


     # compute reciprocal rank scores by Frank Morales 09/05/2024
    for node_with_score in results:
        rank+=1
        if not isinstance(node_with_score, NodeWithScore):
            raise TypeError("node_with_score must be a NodeWithScore object.")
        text = node_with_score.node.get_content()
        text_to_node[text] = node_with_score
        if text not in fused_scores:
          fused_scores[text] = 0.0
        fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes


class RAGFusionPipelinePack(BaseLlamaPack):
    """RAG Fusion pipeline.

    Create a bunch of vector indexes of different chunk sizes.

    """

    def __init__(
        self,
        documents: List[Document],
        llm: Optional[LLM] = None,
        chunk_sizes: Optional[List[int]] = None,
    ) -> None:
        """Init params."""
        self.documents = documents
        self.chunk_sizes = chunk_sizes or DEFAULT_CHUNK_SIZES

        # construct index
        self.llm = llm or OpenAI(model="gpt-3.5-turbo")

        self.query_engines = []
        self.retrievers = {}
        for chunk_size in self.chunk_sizes:
            splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=0)
            nodes = splitter.get_nodes_from_documents(documents)

            service_context = ServiceContext.from_defaults(llm=self.llm)
            vector_index = VectorStoreIndex(nodes, service_context=service_context)
            self.query_engines.append(vector_index.as_query_engine())

            self.retrievers[str(chunk_size)] = vector_index.as_retriever()

        # define rerank component
        rerank_component = FnComponent(fn=reciprocal_rank_fusion)

        # construct query pipeline
        p = QueryPipeline()
        module_dict = {
            **self.retrievers,
            "input": InputComponent(),
            "summarizer": TreeSummarize(),
            # NOTE: Join args
            "join": ArgPackComponent(),
            "reranker": rerank_component,
        }
        p.add_modules(module_dict)
        # add links from input to retriever (id'ed by chunk_size)
        for chunk_size in self.chunk_sizes:
            p.add_link("input", str(chunk_size))
            p.add_link(str(chunk_size), "join", dest_key=str(chunk_size))
        p.add_link("join", "reranker")
        p.add_link("input", "summarizer", dest_key="query_str")
        p.add_link("reranker", "summarizer", dest_key="nodes")

        self.query_pipeline = p

    def get_modules(self) -> Dict[str, Any]:
        """Get modules."""
        return {
            "llm": self.llm,
            "retrievers": self.retrievers,
            "query_engines": self.query_engines,
            "query_pipeline": self.query_pipeline,
        }

    def run(self, *args: Any, **kwargs: Any) -> Any:
        """Run the pipeline."""
        return self.query_pipeline.run(*args, **kwargs)


In [28]:
!pip install llama-index-llms-langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 39.7 MB/s eta 0:00:00


In [29]:
pack = RAGFusionPipelinePack(docs, llm)
query0="What did the author do growing up?"
response0 = pack.run(query=query0)

<ipython-input-27-df6af02ee39d>:91: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=self.llm)


In [30]:
print(response0)

The author, growing up, worked on writing short stories and programming. They wrote simple games, a program to predict rocket heights, and even a word processor. Additionally, the author started programming on a TRS-80 computer and later switched to studying AI in college due to their interest sparked by a novel and a PBS documentary.


# Examples of Queries

In [31]:
#modify By FM 11/01/2024

#response = pack.run(query="What did the author do growing up?")
query0="What did the author do growing up?"
query='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'
query1 = "Who is the President of the USA?"
query2 = "Who is the best poet of CANADA?"
#query2 = "Who won the baseball World Series in 2023? and Who Lost"
query3 = 'Anything about FORTRAN'
query4 = 'Anything about LIPS'
query5 = 'Anything about Python'


response0 = pack.run(query=query0)
response1 = pack.run(query=query1)
response2 = pack.run(query=query2)
response4 = pack.run(query=query4)

print()
print(query0)
print(str(response0))
print()

print()
print(query1)
print(str(response1))
print()

print()
print(query2)
print(str(response2))
print()

print()
print(query4)
print(str(response4))
print()


What did the author do growing up?
The author, growing up, worked on writing short stories and programming. They wrote simple games, a program to predict rocket heights, and a word processor on a TRS-80 computer. Additionally, they took philosophy courses in college before switching to studying AI, inspired by works like Heinlein's "The Moon is a Harsh Mistress" and seeing SHRDLU in action.


Who is the President of the USA?
I am unable to provide real-time information or updates. For the most current information on the President of the USA, please refer to the latest news sources or official government websites.


Who is the best poet of CANADA?
I cannot provide an answer to the query as the information provided does not mention or relate to any specific poet from Canada.


Anything about LIPS
LISP, a programming language, is discussed in the provided context. It is highlighted that LISP is interesting for its own sake and not just for its association with AI. The language was initia